In [2]:
import requests
import json
import time
from datetime import datetime

import sys
import pymysql

import pandas as pd
import numpy as np

In [3]:
def get_eqkmsg(npage, nrows=10):
        endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
        mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
        datatype = 'JSON'
        params = {'ServiceKey':mykey, 'fromTmFc':eqkt, 'toTmFc':eqkt,
                'dataType':datatype, 'pageNo':npage, 'numOfRows':nrows}
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        res = json.loads(r.text).get('response')
        return res

In [4]:
def eqkmsg_count(fromt=None, tot=None, npage=1, nrow=10, gettoday=False):
    global eqkt, rescount, rpages, rrows

    if type(fromt) == int:
        fromt = str(fromt)
    if type(tot) == int:
        tot = str(tot)
    
    today = str(datetime.today())[0:10].split('-')
    eqkt = today[0]+today[1]+today[2]
    if gettoday:
        fromt, tot = eqkt, eqkt
        npage = str(int(npage))
        nrow = str(int(nrow))
    endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
    mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
    datatype = 'JSON'

    if gettoday:
        print ('오늘의 지진 통보를 검색합니다.')
        params = {'ServiceKey':mykey, 'fromTmFc':eqkt, 'toTmFc':eqkt,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    else:
        print ('조회기간: '+fromt[0:4]+'년'+fromt[4:6]+'월'+fromt[6:8]+'일 ~',tot[4:6]+'월'+tot[6:8]+'일')
        params = {'ServiceKey':mykey, 'fromTmFc':fromt, 'toTmFc':tot,
            'dataType':datatype, 'pageNo':npage, 'numOfRows':nrow}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 api 호출에 오류 발생')
        sys.exit(1)

    if r.status_code == 200:
        try:
            res = json.loads(r.text).get('response')
            rcode = res.get('header')['resultCode']
            print('기상청 api 호출 성공 (API result code:',rcode+')')
        except:
            print('통보문 데이터를 불러오는데 실패했습니다.')
            sys.exit(1)

        if rcode == '00':
            rescount = res.get('body')['totalCount']
            rrows = int(res.get('body')['numOfRows'])
            rpages = (rescount//int(nrow)+1)
            if gettoday:
                print('오늘은 지진 통보가',rescount,'개 있습니다.')
            else:
                print('조회기간 동안 지진 통보는 총 ',rescount,'개 입니다.')
        elif rcode == '03':
            if gettoday:
                print('오늘은 지진 통보가 없습니다.')
            else:
                print('조회기간 동안 지진 통보가 없습니다.')

    if r.status_code != 200:
        print('기상청 api 호출 실패 code:',r.status_code)
        sys.exit(1)

    return rescount

In [5]:
def eqk_data():
    global eqk_list
    eqk_dict = {}
    eqk_list = []
    for i in range(0, rpages):
        try:
            r = get_eqkmsg(str(i+1))
            raw = r.get('body').get('items')['item']
        except:
            print('지진 통보문 목록 불러오기 실패')
            break
        for j in range(0, len(raw)):
            eqk_dict = {}
            data = raw[j]
            eqk_dict['id'] = eqkt+'-'+str(i)+str(j)
            eqk_dict['day'] = str(data.get('tmEqk'))[0:7]
            eqk_dict['eqkt'] = str(data.get('tmEqk'))[8:14]
            eqk_dict['fct'] = str(data.get('tmFc'))[8:12]
            eqk_dict['mt'] = data.get('mt')
            eqk_dict['loc'] = data.get('loc')
            eqk_dict['lat'] = data.get('lat')
            eqk_dict['lon'] = data.get('lon')
            eqk_dict['mapURL'] = data.get('img')
            eqk_dict['fctype'] = data.get('fcTp')
            eqk_list.append(eqk_dict)
        time.sleep(1)

    pdcol = eqk_list[0].keys()
    eqk_df = pd.DataFrame(columns=pdcol, data=[eqk_list[i].values() for i in range(0,len(eqk_list))])
    return eqk_df

In [13]:
def main():
    try:
        eqkmsg_count(gettoday=True)
        eqk_data()
    except:
        print('end')

In [14]:
if __name__ == '__main__':
    main()

오늘의 지진 통보를 검색합니다.
기상청 api 호출 성공 (API result code: 03)
오늘은 지진 통보가 없습니다.
end
